# Week 5 Problem Set

## Homework

**HW1.** *Dictionary:* Write two functions:
1. `count_degrees(G)`: which sums up the degrees of all vertices in the graph. The degree of a vertex is defined as the number of edges connected to a Vertex. The argument `G` is a dictionary that represents the graph.
2. `count_edges(G)`: which counts the number of edges in the graph. An edge is defined as a connection between two vertices. The argument `G` is a dictionary.

In [ ]:
def count_degrees(G: dict):
    return sum( len(edge_list) for edge_list in G.values() )


def count_edges(G: dict[str, list]):
    G_copy = G.copy()
    seen_nodes = []
    edges = 0
    for node, edge_list in G_copy.items():
        for connected_node in edge_list:
            if connected_node not in seen_nodes:
                edges += 1
        seen_nodes.append(node)
    return edges


In [ ]:
g1 = {"A": ["B", "E"],
      "B": ["A", "C"],
      "C": ["B", "D", "E"],
      "D": ["C"],
      "E": ["A", "C"]}

d = count_degrees(g1)
e = count_edges(g1)

assert d == 10
assert e == 5

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** Create a class called `GraphSearch` which is a subclass of the class `Graph`. This class should override the method `_create_vertex(id)` and instantiate a `VertexSearch` object instead of `Vertex`.

In [ ]:
class Vertex:
    def __init__(self, id_=""):
        self.id_ = id_
        self.neighbours: dict[Vertex, int] = {}
    
    def add_neighbour(self, nbr_vertex, weight: int = 0):
        self.neighbours[nbr_vertex] = weight
    
    def get_neighbours(self):
        return list(self.neighbours.keys())
    
    def get_weight(self, neighbour):
        return self.neighbours.get(neighbour)
    
    def __eq__(self, other):
        return self.id_ == other.id_
    
    def __lt__(self, other):
        return self.id_ < other.id_
    
    def __hash__(self):
        return hash(self.id_)
    
    def __str__(self):
        return f'Vertex {self.id_} is connected to: {", ".join(neighbour.id_ for neighbour in self.get_neighbours())}'

In [ ]:
import sys

class VertexSearch(Vertex):
    def __init__(self, id_=""):
        super().__init__(id_)
        self.colour: str = "white"
        self.d: int = sys.maxsize
        self.f: int = sys.maxsize
        self.parent: Vertex = None
    



In [ ]:
class Graph:
    def __init__(self):
        self.vertices: dict[str, Vertex] = {}
        
    def _create_vertex(self, id_):
        return Vertex(id_)
    
    def add_vertex(self, id_):
        self.vertices[id_] = self._create_vertex(id_)
    
    def get_vertex(self, id_):
        return self.vertices.get(id_)
    
    def add_edge(self, start_v: str, end_v: str, weight=0):
        if self.get_vertex(start_v) is None:
            raise ValueError('start_v was not in graph')
        elif self.get_vertex(end_v) is None:
            raise ValueError('end_v was not in graph')

        self.get_vertex(start_v).add_neighbour(self.get_vertex(end_v), weight)

    def get_neighbours(self, id_):
        if self.get_vertex(id_) is None:
            return None
        return [neighbour.id_ for neighbour in self.get_vertex(id_).get_neighbours()]
    
    def __contains__(self, val):
        return self.get_vertex(val) is not None
    
    def __iter__(self):
        for k,v in self.vertices.items():
            yield v 
        
    # write a code to create a computed property called num_vertices
    @property
    def num_vertices(self):
        return len(self.vertices)


In [ ]:
class GraphSearch(Graph):
    def _create_vertex(self, id_):
        return VertexSearch(id_)

In [ ]:
g2 = GraphSearch()
g2.add_vertex("Z")
assert(type(g2.vertices["Z"]) == type(VertexSearch()))

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** *Undirected Graph:* **You need to complete CS5 and CS6 before you do this homework.** Paste your answer for `Search2D` and `SearchBFS` classes into the cell below.

Create a subclass of `GraphSearch` class called `UGraphSearch` for undirected graphs. All edges in undirected graphs are bidirectional (i.e. vertex1 <-> vertex2). 
This means that you need to override the `add_edge()` method.

In [ ]:
class Stack:
    def __init__(self):
        self._items = []
        
    def push(self, item):
        self._items.append(item)

    def pop(self):
        if self.is_empty:
            return None
        return self._items.pop()

    def peek(self):
        if self.is_empty:
            return None
        return self._items[-1]

    @property
    def is_empty(self):
        return self.size == 0

    @property
    def size(self):
        return len(self._items)

class Queue:
    def __init__(self):
        self.left_stack = Stack()
        self.right_stack = Stack()
    
    def enqueue(self, new_item):
        self.right_stack.push(new_item)

    def dequeue(self):
        if self.left_stack.is_empty:
            self.left_stack._items = self.right_stack._items[::-1]
            self.right_stack._items.clear()
        return self.left_stack.pop()

    def peek(self):
        if self.is_empty:
            return None
        elif self.left_stack.is_empty:
            return self.right_stack._items[0]
        else:
            return self.left_stack.peek()
    
    @property
    def is_empty(self):
        return self.left_stack.is_empty and self.right_stack.is_empty
    
    @property
    def size(self):
        return self.left_stack.size + self.right_stack.size
        


In [ ]:
import sys

class Search2D:
    def __init__(self, g: Graph):
        self.graph = g
    
    def clear_vertices(self):
        for vertex in self.graph.vertices.values():
            vertex.colour = "white"
            vertex.d = sys.maxsize
            vertex.f = sys.maxsize
            vertex.parent = None
    
    def __iter__(self):
        return iter([v for v in self.graph])
    
    def __len__(self):
        return len([v for v in self.graph.vertices])

In [ ]:
class SearchBFS(Search2D):

    def search_from(self, start: str):
        self.clear_vertices()
        search_queue = Queue()
        start_vertex = self.graph.get_vertex(start)
        start_vertex.d = 0
        start_vertex.colour = 'black'
        search_queue.enqueue(start_vertex)
        while not search_queue.is_empty:
            current_vertex: VertexSearch = search_queue.dequeue()
            print(f'current_vertex = {current_vertex}')
            for neighbour in current_vertex.neighbours:
                if neighbour.colour == 'black': continue
                
                neighbour.colour = 'black'
                neighbour.parent = current_vertex
                neighbour.d = current_vertex.d + 1
                search_queue.enqueue(neighbour)
    
    def get_shortest_path(self, start, dest):
        res = []
        self.get_path(start, dest, res)
        if res is None or len(res) == 0:
            return None
        return res
    
    def get_path(self, start: str, dest: str, result: list) -> list | None:
        if self.graph.get_vertex(start) is None or self.graph.get_vertex(dest) is None:
            return None
        if len(result) == 0:
            self.search_from(start)
            result.append(dest)
        
        start_vertex: VertexSearch = self.graph.get_vertex(start)
        end_vertex: VertexSearch = self.graph.get_vertex(dest)

        if start_vertex == end_vertex:
            # we're done!
            return
        elif end_vertex.parent is None:
            result.insert(0, "No Path")
        else:
            result.insert(0, end_vertex.parent.id_)
            self.get_path(start, end_vertex.parent.id_, result)

        

In [ ]:
class UGraphSearch(GraphSearch):
    def add_edge(self, start_v, end_v, weight=0):
        if self.get_vertex(start_v) is None:
            raise ValueError('start_v was not in graph')
        elif self.get_vertex(end_v) is None:
            raise ValueError('end_v was not in graph')
        self.get_vertex(start_v).add_neighbour(self.get_vertex(end_v))
        self.get_vertex(end_v).add_neighbour(self.get_vertex(start_v))

In [ ]:
g2 = UGraphSearch()
assert g2.vertices == {} and g2.num_vertices == 0
g2.add_vertex("L")
g2.add_vertex("I")
g2.add_vertex("S")
g2.add_vertex("P")
assert g2.num_vertices == 4
assert "L" in g2
assert "I" in g2
assert "S" in g2
assert "P" in g2
g2.add_edge("L", "I")
g2.add_edge("I", "S")
g2.add_edge("S", "P")
assert sorted(g2.get_neighbours("L")) == ["I"]
assert sorted(g2.get_neighbours("I")) == ["L", "S"]
assert sorted(g2.get_neighbours("S")) == ["I", "P"]
assert sorted(g2.get_neighbours("P")) == ["S"]

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
g2 = UGraphSearch()
g2.add_vertex("r")
g2.add_vertex("s")
g2.add_vertex("t")
g2.add_vertex("u")
g2.add_vertex("v")
g2.add_vertex("w")
g2.add_vertex("x")
g2.add_vertex("y")
g2.add_vertex("z")
g2.add_edge("r", "s")
g2.add_edge("r", "v")
g2.add_edge("s", "w")
g2.add_edge("t", "u")
g2.add_edge("t", "x")
g2.add_edge("t", "w")
g2.add_edge("u", "t")
g2.add_edge("u", "x")
g2.add_edge("u", "y")
g2.add_edge("v", "r")
g2.add_edge("w", "s")
g2.add_edge("w", "t")
g2.add_edge("w", "x")
g2.add_edge("x", "w")
g2.add_edge("x", "t")
g2.add_edge("x", "u")
g2.add_edge("x", "y")
g2.add_edge("y", "u")
g2.add_edge("y", "x")
gs = SearchBFS(g2)
gs.search_from("s")
assert gs.graph.get_vertex("s").d == 0
assert gs.graph.get_vertex("t").d == 2
assert gs.graph.get_vertex("u").d == 3
assert gs.graph.get_vertex("v").d == 2
assert gs.graph.get_vertex("w").d == 1
assert gs.graph.get_vertex("x").d == 2
assert gs.graph.get_vertex("y").d == 3
assert gs.graph.get_vertex("r").d == 1
ans = gs.get_shortest_path("s", "u")
assert ans == ["s", "w", "t", "u"] or ans == ["s", "w", "x", "u"]
ans = gs.get_shortest_path("v", "s")
assert ans == ["v", "r", "s"]
ans = gs.get_shortest_path("v", "y")
assert ans == ["v", "r", "s", "w", "x", "y"]

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW4.** *Depth-First-Search:* Create a class `SearchDFS` as a child class of `Search2D` to implement Depth-First-Search algorithm. You should add one additional attribute:
- `time`: which is an attribute to record the discovery time and the finishing time.

The class should also implement the following methods:
- `search()`: which modifies the vertices' properties such as `colour`, `d`, and `parent` following Depth-First-Search algorithm.
- `dfs_visit(vert)`: which is the recursive method for visiting a vertex in Depth-First-Search algorithm.

In [ ]:
import sys

class SearchDFS(Search2D):
    def __init__(self, g: GraphSearch):
        self.graph = g
        self.time = 0
      
    def search(self):
        vertices = list(self.graph.vertices.values())
        for vertex in vertices:
            if vertex.colour != "black":
                self.dfs_visit(vertex)
    
    def dfs_visit(self, vert: VertexSearch):
        self.time += 1
        vert.colour = "black"
        print(f"exploring vertex {vert}")
        vert.d = self.time
        for child in vert.get_neighbours():
            if child.colour != "black":
                child.parent = vert
                self.dfs_visit(child)
        vert.colour = "black"
        self.time += 1
        vert.f = self.time


In [ ]:
g4 = GraphSearch()
g4.add_vertex("e")
g4.add_vertex("m")
g4.add_vertex("a")
g4.add_vertex("c")
g4.add_vertex("s")
g4.add_edge("e", "m")
g4.add_edge("m", "a")
g4.add_edge("a", "c")
g4.add_edge("c", "s")
gs = SearchDFS(g4)
gs.search()
assert gs.graph.get_vertex("e").parent == None 
assert gs.graph.get_vertex("m").parent == gs.graph.get_vertex("e")
print(gs.graph.get_vertex("a").f)
assert gs.graph.get_vertex("m").d == 2 and gs.graph.get_vertex("a").f == 8
assert gs.graph.get_vertex("c").d == 4 and gs.graph.get_vertex("s").f == 6

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW5.** *Topological Sort:* Create a function that takes in a `SearchDFS` object to perform a topological sort:
- `topological_sort(g)`: which takes in a `SearchDFS` object and returns a list of `VertexSearch` objects sorted based on their `f` attribute. This method should call the `search()` method of the `SearchDFS` to first calculate the `f` attribute of the vertices. Note that you need to copy the `SearchDFS` object of your input argument so as not to mutate the object.

In [ ]:
import sys
import copy

def topological_sort(g: SearchDFS):
    new_graph = copy.deepcopy(g)
    new_graph.search()
    return reversed(sorted(new_graph.graph.vertices.values(), key=lambda vertex: vertex.f))

In [ ]:
import copy
g = GraphSearch()
g.add_vertex("3/4 cup milk")
g.add_vertex("1 egg")
g.add_vertex("1 tbl oil")
g.add_vertex("1 cup mix")
g.add_vertex("heat syrup")
g.add_vertex("heat griddle")
g.add_vertex("pour 1/4 cup")
g.add_vertex("turn when bubbly")
g.add_vertex("eat")
g.add_edge("heat griddle", "pour 1/4 cup")
g.add_edge("3/4 cup milk", "1 cup mix")
g.add_edge("1 egg", "1 cup mix")
g.add_edge("1 tbl oil", "1 cup mix")
g.add_edge("1 cup mix", "heat syrup")
g.add_edge("1 cup mix", "pour 1/4 cup")
g.add_edge("pour 1/4 cup", "turn when bubbly")
g.add_edge("turn when bubbly", "eat")
g.add_edge("heat syrup", "eat")
gs = SearchDFS(g)  

path = topological_sort(gs)
ans = [v.f for v in copy.copy(path)]
print(ans)
assert ans == [18, 16, 14, 12, 11, 10, 9, 6, 5]
ans = [v.id_ for v in copy.copy(path)]
assert ans == ['heat griddle', '1 tbl oil', '1 egg', '3/4 cup milk', '1 cup mix', 'pour 1/4 cup', 'turn when bubbly', 'heat syrup', 'eat']

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
